In [1]:
import pandas as pd
import matplotlib.pyplot as plt

### Tratamento de dados ###

In [2]:
df = pd.read_csv('zap_imoveis.csv', index_col='id')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3944 entries, 0 to 3943
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   condominio   3944 non-null   float64
 1   anunciante   3944 non-null   object 
 2   num_suites   3944 non-null   float64
 3   num_vagas    3944 non-null   float64
 4   num_quartos  3944 non-null   float64
 5   area         3944 non-null   float64
 6   rua          2932 non-null   object 
 7   regiao       3944 non-null   object 
 8   descricao    3764 non-null   object 
 9   tipo_imovel  3944 non-null   object 
 10  localidade   3944 non-null   object 
 11  bairro       3944 non-null   object 
 12  preco        3944 non-null   int64  
dtypes: float64(5), int64(1), object(7)
memory usage: 431.4+ KB


#### limpeza da base

In [3]:
df = df.drop(['anunciante','descricao','localidade','rua','regiao','bairro'], axis = 1)
df = df.query("tipo_imovel == 'Apartamento' or tipo_imovel == 'Casa' or tipo_imovel == 'Casa de Condomínio'")
df.head().T

id,0,1,2,3,4
condominio,0.0,0.0,18.0,230.0,680.0
num_suites,2.0,0.0,0.0,0.0,1.0
num_vagas,2.0,0.0,1.0,1.0,1.0
num_quartos,3.0,0.0,2.0,2.0,3.0
area,150.0,360.0,45.0,55.0,147.0
tipo_imovel,Casa,Casa,Casa,Casa,Casa
preco,570000,200000,170000,250000,600000


In [4]:
df['tipo_imovel'].value_counts()

Apartamento           1623
Casa                  1502
Casa de Condomínio     150
Name: tipo_imovel, dtype: int64

### Definição de dados de para treino e teste

In [5]:
from sklearn.model_selection import train_test_split

X = df.drop('tipo_imovel',axis=1)
y = df['tipo_imovel']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

### Ajuste na escala dos dados

In [6]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()  
scaler.fit(X_train)
X_train = scaler.transform(X_train)  
X_test = scaler.transform(X_test)

## KNN

#### Treino do modelo

In [7]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=3)

knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

#### Predição

In [8]:
resultado_knn = knn.predict(X_test)

#### Matriz de confusão

In [9]:
print (pd.crosstab(y_test,resultado_knn, rownames=['Real'], colnames=['Predicted'], margins=True))

Predicted           Apartamento  Casa  Casa de Condomínio  All
Real                                                          
Apartamento                 417    84                   4  505
Casa                         77   350                   5  432
Casa de Condomínio           22    15                   9   46
All                         516   449                  18  983


#### Relatório da classificação KNN

In [10]:
from sklearn import metrics

print(metrics.classification_report(y_test,resultado_knn))

                    precision    recall  f1-score   support

       Apartamento       0.81      0.83      0.82       505
              Casa       0.78      0.81      0.79       432
Casa de Condomínio       0.50      0.20      0.28        46

          accuracy                           0.79       983
         macro avg       0.70      0.61      0.63       983
      weighted avg       0.78      0.79      0.78       983



#### Verificação de ajustes para o modelo ###

In [11]:
from sklearn.model_selection import GridSearchCV

#adicionando a quantidade de valores para k (número de vizinhos) a um dicionário
k_list = list(range(1,31))
parametros = dict(n_neighbors=k_list)

#passando nosso modelo, os valores para k, dobras pra validação cruzada e o scoring para o objeto GridSearch 
grid = GridSearchCV(knn, parametros, cv=5, scoring='accuracy')

#treinamento
grid.fit(df.drop('tipo_imovel',axis=1), df['tipo_imovel'])

print("Melhor parâmetro {} com acurácia de {:.2%} ".format(grid.best_params_,grid.best_score_))

Melhor parâmetro {'n_neighbors': 3} com acurácia de 72.82% 


## Decision Tree


#### Treinamentodo modelo

In [12]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(criterion="entropy")

clf = clf.fit(X_train,y_train)

#### Verificação das features mais importantes

In [13]:
for feature,importancia in zip(df.columns,clf.feature_importances_):
    print("{}: {:.2%}".format(feature,importancia))

condominio: 30.69%
num_suites: 4.80%
num_vagas: 7.60%
num_quartos: 3.44%
area: 27.94%
tipo_imovel: 25.52%


#### Predição

In [14]:
resultado_clf = clf.predict(X_test)

#### Matriz de confusão

In [15]:
print (pd.crosstab(y_test,resultado_clf, rownames=['Real'], colnames=['Predicted'], margins=True))

Predicted           Apartamento  Casa  Casa de Condomínio  All
Real                                                          
Apartamento                 401    80                  24  505
Casa                         68   354                  10  432
Casa de Condomínio           16    17                  13   46
All                         485   451                  47  983


#### Relatório da classificação Árvore de Decisão

In [16]:
print(metrics.classification_report(y_test,resultado_clf))

                    precision    recall  f1-score   support

       Apartamento       0.83      0.79      0.81       505
              Casa       0.78      0.82      0.80       432
Casa de Condomínio       0.28      0.28      0.28        46

          accuracy                           0.78       983
         macro avg       0.63      0.63      0.63       983
      weighted avg       0.78      0.78      0.78       983

